# Faiss

>[Facebook AI Similarity Search (FAISS)](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also includes supporting code for evaluation and parameter tuning.
>
>See [The FAISS Library](https://arxiv.org/pdf/2401.08281) paper.

You can find the FAISS documentation at [this page](https://faiss.ai/).

This notebook shows how to use functionality related to the `FAISS` vector database. It will show functionality specific to this integration. After going through, it may be useful to explore [relevant use-case pages](/docs/how_to#qa-with-rag) to learn how to use this vectorstore as part of a larger chain.

## Setup

The integration lives in the `langchain-community` package. We also need to install the `faiss` package itself. We can install these with:

Note that you can also install `faiss-gpu` if you want to use the GPU enabled version

In [6]:
pip install -qU langchain-community faiss-cpu

In [9]:
!pip install -U "requests==2.32.4"
!pip install -U langchain langchain-community langchain-openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 532.2 kB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.5
    Uninstalling requests-2.32.5:
      Successfully uninstalled requests-2.32.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.29 requires requests<3,>=2.32.5, but you have requests 2.32.4 which is incompatible.
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 532.0 kB/s eta 0:00:00
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

If you want to get best in-class automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [4]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## Initialization

import EmbeddingTabs from "@theme/EmbeddingTabs";

<EmbeddingTabs/>


In [15]:
import os
from getpass import getpass

# OpenAI APIキーを入力（非表示で入力できる）
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

Enter your OpenAI API key: ··········


In [16]:
# | output: false
# | echo: false
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [17]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

## Manage vector store

### Add items to vector store

In [18]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['cb295199-f10c-4fa3-8088-094de9c95b88',
 '85b0e6c9-d4be-4622-a250-a83fb5075821',
 '542bee27-19af-4d59-bb28-66ce5aeb8fcd',
 'd62b0ad3-b676-4393-a542-214edb77f887',
 '2446689c-34d7-465a-8246-2fc33c997668',
 '40ae43ea-ad4b-4f19-b6be-d966390d24e1',
 'c51b1134-0705-4bfb-96a3-a2dd33511fec',
 '1731cb27-567c-4898-99ad-a7a6d7f38c6b',
 '0c567ff8-8927-4262-899f-56e902b3c91f',
 '9fd6b1b7-35f9-427f-a754-f8a4de43ca11']

### Delete items from vector store

In [19]:
vector_store.delete(ids=[uuids[-1]])

True

## Query vector store

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent.

### Query directly

#### Similarity search

Performing a simple similarity search with filtering on metadata can be done as follows:

In [20]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


Some [MongoDB query and projection operators](https://www.mongodb.com/docs/manual/reference/operator/query/) are supported for more advanced metadata filtering. The current list of supported operators are as follows:
- `$eq` (equals)
- `$neq` (not equals)
- `$gt` (greater than)
- `$lt` (less than)
- `$gte` (greater than or equal)
- `$lte` (less than or equal)
- `$in` (membership in list)
- `$nin` (not in list)
- `$and` (all conditions must match)
- `$or` (any condition must match)
- `$not` (negation of condition)

Performing the same above similarity search with advanced metadata filtering can be done as follows:

In [21]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": {"$eq": "tweet"}},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


#### Similarity search with score

You can also search with score:

In [22]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.893776] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


#### Other search methods


There are a variety of other ways to search a FAISS vector store. For a complete list of those methods, please refer to the [API Reference](https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html)

### Query by turning into retriever

You can also transform the vector store into a retriever for easier usage in your chains.

In [23]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(id='d62b0ad3-b676-4393-a542-214edb77f887', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

## Usage for retrieval-augmented generation

For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:

- [Tutorials](/docs/tutorials/rag)
- [How-to: Question and answer with RAG](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [Retrieval conceptual docs](https://python.langchain.com/docs/concepts/retrieval)

## Saving and loading
You can also save and load a FAISS index. This is useful so you don't have to recreate it everytime you use it.

In [24]:
vector_store.save_local("faiss_index")

new_vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

docs = new_vector_store.similarity_search("qux")

In [25]:
docs[0]

Document(id='542bee27-19af-4d59-bb28-66ce5aeb8fcd', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!')

## Merging
You can also merge two FAISS vectorstores

In [26]:
db1 = FAISS.from_texts(["foo"], embeddings)
db2 = FAISS.from_texts(["bar"], embeddings)

db1.docstore._dict

{'56f83227-28ce-415c-9fe1-6cd26d020327': Document(id='56f83227-28ce-415c-9fe1-6cd26d020327', metadata={}, page_content='foo')}

In [27]:
db2.docstore._dict

{'730e66aa-88c1-4322-8d24-824f420caca4': Document(id='730e66aa-88c1-4322-8d24-824f420caca4', metadata={}, page_content='bar')}

In [28]:
db1.merge_from(db2)

In [29]:
db1.docstore._dict

{'56f83227-28ce-415c-9fe1-6cd26d020327': Document(id='56f83227-28ce-415c-9fe1-6cd26d020327', metadata={}, page_content='foo'),
 '730e66aa-88c1-4322-8d24-824f420caca4': Document(id='730e66aa-88c1-4322-8d24-824f420caca4', metadata={}, page_content='bar')}

## API reference

For detailed documentation of all `FAISS` vector store features and configurations head to the API reference: https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html